# A Tutorial on Community Models and Integrations - PhysicsNeMo

In this tutorial, PhysicsNeMo is used to expand the scope of community models and datasets, such as [The Well](https://github.com/PolymathicAI/the_well) by leveraging physics informed utilities, optimized model layers, and MLOps best practices. Specifically, the tutorial covers:
. Loading and evaluating a checkpoint from The Well 
. How to use a pretrained checkpoint from The Well and run it as a PhysicsNeMo user
. Training community models with PhysicsNeMo
. Fine-tuning community models with PhysicsNeMo
. Experimenting with different architectures, from the community and internal to PhysicsNeMo

## Loading Models and Data from The Well

To begin, a model and dataset from The Well is selected for use throughout this example. For this example, the [Magnetohydrodynamics dataset](https://github.com/PolymathicAI/the_well/tree/master/datasets/MHD_64) is used. Magnetohydrodynamics (MHD), is the study of the dynamics of electrically conducting fluids such as plasmas.

Note that any one of the models and dataset combinations may be selected.

In addition to the data, a pre-trained model with a Tucker-Factorized Fourier Neural Operator (TFNO) architecture is used that will later be converted to PhysicsNeMo.

The dataset streaming functionality from The Well can be utilized so that the dataset does not need to be downloaded locally. This requires accessing huggingface. Alternatively, the dataset can be downloaded which takes around 71GB of storage. The command to download all splits is `the-well-download --base-path path/to/base --dataset MHD_64`


In [ ]:
from the_well.benchmark.models import TFNO
from torchinfo import summary

# Load The Well model
well_model = TFNO.from_pretrained("polymathic-ai/TFNO-MHD_64")

# Have a look at the model summary
# Note that the order of layers does not represent order of execution
summary(well_model, depth=5)

In [ ]:
from the_well.data import WellDataset
from the_well.data.normalization import ZScoreNormalization

# Enable streaming the dataset from HuggingFace or loading from local directory
# The following line may take a couple of minutes to instantiate the datamodule
dataset = WellDataset(
    # well_base_path="hf://datasets/polymathic-ai/",  # access from HF hub, or locally.
    well_base_path="./TheWellMHDData/datasets",
    well_dataset_name="MHD_64",
    well_split_name="train",
    use_normalization=True,
    normalization_type=ZScoreNormalization,
    n_steps_input=4,
    n_steps_output=1,
)

With the dataset on hand, its features, shape and size can be explored. `The Well` provides a great script for this already, and is left to the reader to go through if desired. A summary is provided below, also available on the [dataset card](https://github.com/PolymathicAI/the_well/blob/master/datasets/MHD_64/README.md) online:

**Dimension of discretized data:** 100 timesteps of 64 $\times$ 64 $\times$ 64 cubes.

**Fields available in the data:** Density (scalar field), velocity (vector field), magnetic field (vector field).

**Number of trajectories:** 10 Initial conditions x 10 combination of parameters = 100 trajectories.

**Estimated size of the ensemble of all simulations:** 71.6 GB.

**Grid type:** uniform grid, cartesian coordinates.

**Initial conditions:** uniform IC.

**Boundary conditions:** periodic boundary conditions.

**Data are stored separated by ($\Delta t$):** 0.01 (arbitrary units).

**Total time range ($t\_{min}$ to $t\_{max}$):** $t\_{min} = 0$, $t\_{max} = 1$.

**Spatial domain size ($L_x$, $L_y$, $L_z$):** dimensionless so 64 pixels.

**Set of coefficients or non-dimensional parameters evaluated:** all combinations of $\mathcal{M}_s=${0.5, 0.7, 1.5, 2.0 7.0} and $\mathcal{M}_A =${0.7, 2.0}.

**Approximate time and hardware used to generate the data:** Downsampled from `MHD_256` after applying ideal low-pass filter.

**What phenomena of physical interest are catpured in the data:** MHD fluid flows in the compressible limit (sub and super sonic, sub and super Alfvenic).

**How to evaluate a new simulator operating in this space:** Check metrics such as Power spectrum, two-points correlation function.

Please cite the associated paper if you use this data in your research:

```
@article{burkhart2020catalogue,
  title={The catalogue for astrophysical turbulence simulations (cats)},
  author={Burkhart, B and Appel, SM and Bialy, S and Cho, J and Christensen, AJ and Collins, D and Federrath, Christoph and Fielding, DB and Finkbeiner, D and Hill, AS and others},
  journal={The Astrophysical Journal},
  volume={905},
  number={1},
  pages={14},
  year={2020},
  publisher={IOP Publishing}
}
```




A single sample can be extracted and inspected. Some notes from `The Well` are shown below. More info on examining their data can be found in [this example](https://github.com/PolymathicAI/the_well/blob/master/docs/tutorials/dataset.ipynb):

The most important elements are `input_fields` and `output_fields`. They represent the time-varying physical fields of the dynamical system and are generally the input and target of our models. For a dynamical system that has 2 spatial dimensions $x$ and $y$, `input_fields` would have a shape $(T_{in}, L_x, L_y, F)$ and `output_fields` would have a shape $(T_{out}, L_x, L_y, F)$. The number of input and output timesteps $T_{in}$ and $T_{out}$ are specified at the instantiation of the dataset with the arguments `n_steps_input` and `n_steps_output`. $L_x$ and $L_y$ are the lengths of the spatial dimensions. $F$ represents the number of physical fields, where vector fields $v = (v_x, v_y)$ and tensor fields $t = (t_{xx}, t_{xy}, t_{yx}, t_{yy})$ are flattened.

Note that the dataset for MHD has three spatial dimensions, so an extra $z$ term will be included.

In [ ]:
sample = dataset[0]
for k, v in sample.items():
    print(f"Key: {k.ljust(20)} Shape: {v.shape}")

print(f"Field Names: {dataset.metadata.field_names}")

Using the model summary from above and the order of operations in the [TFNO forward pass](https://github.com/neuraloperator/neuraloperator/blob/7bb578df787d6ca548b623b83f0601c98dc931fb/neuralop/models/fno.py#L337), the data is processed by:

1. Applying optional positional encoding
2. Sending inputs through a lifting layer to a high-dimensional latent space
3. Applying optional domain padding to high-dimensional intermediate function representation
4. Applying `n_layers` Fourier/TFNO layers in sequence (SpectralConvolution + skip connections, nonlinearity) 
5. If domain padding was applied, domain padding is removed
6. Projection of intermediate function representation to the output channels


This pretrained model is trained to predict the $T_{out} = 1$ next states given the $T_{in} = 4$ previous states. The input steps are concatenated along their channels, such that the model expects $T_{in} \times F$ channels as input and $T_{out} \times F$ channels as output. Because `WellDataset` is a PyTorch dataset, we can use it conveniently with PyTorch data-loaders.

We can now evaluate and verify the performacne of the pretrained MHD model from `the_well`. To do this, we will utilize the _streaming_ data mode from `the_well`, and the utilities in the `Trainer` for validation. 

## Using The Well Data to Train PhysicsNeMo Models
Now that the baseline model from The Well has been evaluated, lets look at training a similar architecture from scratch in `PhysicsNeMo`. In the `PhysicsNeMo` examples, there is a similar example of a Tensor-Factorized Fourier Neural Operator implementation that has been used in a related use case for incompressible magnetohydrodynamics in 2D. This model can be adapted for use in 3D with the dataset from The Well. This also serves as an example for how `PhysicsNeMo` can be used as a framework for building and testing model architectures that are not found in base model collection. For the remainder of this notebook, a boiler-plate `Trainer` class is implemented in `utils.py` that contains the core components needed for loading models, running training loops, saving checkpoints, evaluating models, etc. The method `setup_model` needs to be implemented in order to use a model from PhysicsNeMo. For completeness, the TFNO architecture from `PhysicsNeMo` is included in the `tfno` folder. Additionally, there is a config file in `config` folder that is used to initialize some parameters for our pipeline.


In [ ]:
%reload_ext autoreload
import torch

torch.cuda.empty_cache()

In [ ]:
import hydra
from hydra import compose, initialize
from training_utils import Trainer

hydra.core.global_hydra.GlobalHydra.instance().clear()

initialize(version_base=None, config_path="./config")
cfg = compose(config_name="mhd_config.yaml")
cfg.output_dir = "./TEMP"
cfg.train_params.epochs = 1

In [ ]:
class MHDTrainer(Trainer):
    def setup_model(self):
        """Setup the TFNO model based in PhysicsNeMo. Input parameters are set and controlled by the yaml config."""
        from tfno.tfno import TFNO

        self.model = TFNO(
            in_channels=self.model_params.in_dim,
            coord_features=self.model_params.coord_features,
            out_channels=self.model_params.out_dim,
            decoder_layers=self.model_params.decoder_layers,
            decoder_layer_size=self.model_params.fc_dim,
            dimension=self.model_params.dimension,
            latent_channels=self.model_params.layers,
            num_fno_layers=self.model_params.num_fno_layers,
            num_fno_modes=self.model_params.modes,
            padding=[
                self.model_params.pad_z,
                self.model_params.pad_y,
                self.model_params.pad_x,
            ],
            rank=self.model_params.rank,
            factorization=self.model_params.factorization,
            fixed_rank_modes=self.model_params.fixed_rank_modes,
            decomposition_kwargs=self.model_params.decomposition_kwargs,
        ).to(self.dist.device)

        print(summary(self.model, depth=3))

In [ ]:
cfg.train_params.ckpt_path = "./TEMP/checkpoints/pnm_model_well_data"
mhd_trainer = MHDTrainer(cfg)
mhd_trainer.train()

## Running Models From The Well in PhysicsNemo

PhysicsNeMo offers great utility in running community models natively in PhysicsNeMo by way of a simple conversion. The documentation for this can be found [here](https://docs.nvidia.com/deeplearning/physicsnemo/physicsnemo-core/api/physicsnemo.models.html#converting-pytorch-models-to-physicsnemo-models). The first process of the conversion is setting up the base PyTorch model as a PhysicsNeMo model.


In [ ]:
from physicsnemo.registry import ModelRegistry

model_registry = ModelRegistry()
model_registry.__clear_registry__()
model_registry.__restore_registry__()


from dataclasses import dataclass

from physicsnemo.models.meta import ModelMetaData
from physicsnemo.models.module import Module
from the_well.benchmark.models import TFNO


@dataclass
class WellMetaData(ModelMetaData):
    name: str = "WellTFNOModel"


well_nemo_model = Module.from_torch(TFNO, meta=WellMetaData())
print(well_nemo_model)

In [ ]:
from physicsnemo.registry import ModelRegistry

print(well_model.__dict__)
well_nemo_model = ModelRegistry().factory("TFNOPhysicsNeMoModel")(
    dim_in=28,
    dim_out=7,
    n_spatial_dims=3,
    spatial_resolution=[64, 64, 64],
    hidden_channels=128,
    modes1=16,
    modes2=16,
    modes3=16,
)

summary(well_nemo_model, depth=5)

The resulting `well_nemo_model` is simply the underlying PyTorch model from The Well, that can now be used with PhysicsNeMo and its utilities. Note that this is a newly instantiated model. To now train this model in PhysicsNeMo with the same dataset from The Well, we can update our `Trainer` to use this new model. 

In [ ]:
class MHDTrainer(Trainer):
    def setup_model(self):
        """Setup the TFNO model based in PhysicsNeMo. Input parameters are set and controlled by the yaml config."""
        from dataclasses import dataclass

        from physicsnemo.models.meta import ModelMetaData
        from physicsnemo.models.module import Module
        from the_well.benchmark.models import TFNO

        @dataclass
        class WellMetaData(ModelMetaData):
            name: str = "WellTFNOModel"

        well_nemo_model = Module.from_torch(TFNO, meta=WellMetaData())
        well_nemo_model = well_nemo_model(
            dim_in=28,
            dim_out=7,
            n_spatial_dims=3,
            spatial_resolution=[64, 64, 64],
            hidden_channels=128,
            modes1=16,
            modes2=16,
            modes3=16,
        )

        self.model = well_nemo_model.to(self.dist.device)
        print(summary(self.model, depth=5))


cfg.train_params.ckpt_path = "./checkpoints/well_model_well_data"
well_model_trainer = MHDTrainer(cfg)
well_model_trainer.train()

## Fine-tuning Models from The Well in PhysicsNeMo
Using the approach of converting models to PhysicsNeMo allows users to utilize all of the helpful features inside of PhysicsNeMo itself. One great feature of The Well is the abundance of pre-trained model that they provided, typically offering [many pretrained model architectures](https://huggingface.co/collections/polymathic-ai/the-well-benchmark-models-67e69bd7cd8e60229b5cd43e) for a single dataset. These pretrained models can be fine-tuned using PhysicsNeMo by leveraging a similar approach to the above example of converting a model into PhysicsNeMo format. In this section, the pretrained TFNO model will be loaded from The Well, converted to a PhysicsNeMo model, and then fine-tuned on a new dataset.

The new dataset will come from a HuggingFace [community challenge focused around Stellerator design](https://huggingface.co/blog/cgeorgiaw/constellaration-fusion-challenge). In the field of magnetically confined fusion, the physics of magnetohydrodynamics influence the system in many ways, and modeling these equations help researchers and engineers understand reaction ability, equipment design, and plasma shapes, to name a few. In the remainder of this tutorial, we will set up the trainer, inspect the Stellerator desgin dataset, and fine-tune the model!

### Loading The Well Pre-Trained Checkpoint as PhysicsNeMo Model

In [ ]:
class MHDTrainer(Trainer):
    def setup_model(self):
        """Setup the pretrained TFNO model from The Well as a PhysicsNeMo model.
        Model input parameters are set from the well, and the original config is updated to reflect this."""
        import inspect

        from physicsnemo.models.meta import ModelMetaData
        from physicsnemo.models.module import Module
        from the_well.benchmark.models import TFNO

        well_model = TFNO.from_pretrained("polymathic-ai/TFNO-MHD_64")
        model_dict = well_model.__dict__

        signature = inspect.signature(TFNO)
        parameters = signature.parameters
        filtered_params = {k: model_dict[k] for k in parameters if k in model_dict}

        model = Module.from_torch(TFNO, meta=ModelMetaData(name="converted_tfno"))
        well_pretrained_model = model(**filtered_params)
        well_pretrained_model.inner_model.load_state_dict(
            well_model.state_dict(), strict=True
        )
        self.model = well_pretrained_model.to(self.dist.device)

        print(summary(self.model, depth=4))
        print(self.model)


well_model_trainer = MHDTrainer(cfg)
# well_model_trainer.train()

### Exploring the HuggingFace Stellerator Design Dataset
With a simple framework for using community datasets and models with PhysicsNeMo, some deeper problems can be explored. In this section, the pretrained TFNO model for magnetohydrodynamics will be augmented, and used as a foundation for transfer learning onto a new dataset with a related objective: optimizing the design of a stellarator. Stellarators are a type of magnetically confined fusion device for containing plasma of very high temperature and pressure. The design of these devices is complex due to thier geometry. A recent challenge on HuggingFace serves as the basis for the problem.


From the challenge page:

The ConStellaration dataset contains over 150,000 QI equilibria produced by VMEC++. As a reminder, QI stellarators are a subset of configurations that minimize the internal plasma currents that can lead to disruptive events in a tokamak. The provided equilibria correspond to different 3D plasma boundary surfaces and offer samples across a wide and physically meaningful range of parameters. The dataset includes:

Input parameters: the 3D plasma boundary, together with the pressure and current profiles.
Equilibrium outputs: full VMEC++ equilibrium solution plus additional metrics of interest for stellarator design (e.g., degree of QI symmetry, turbulent transport geometrical quantities).

There are many ways in which to use the dataset, and namely there are three unique challenges present that are available to solve, starting with low complexity and moving up to multimodal optimization problems. While this example will not directly solve any of the stated challenges, it will serve as a starting point towards utilizing community models and datasets with PhysicsNeMo, to solve real world problems. 

The goal of the model in this case is to serve as a surrogate for approximating the equilibrium dynamics of the plasma, which then maps to quantities of interest. The dataset contains input geometry and a large list of output parameters that can be learned. Because the pretrained TFNO model already has learned knowledge of magnetohydrodynamics, it is used to translate the input design parameters into predicted properties, by way of mapping input geometry to predicted MHD field, and then onto the properties of interest. The model here will utilize the pretrained TFNO as a frozen intermediate representation, while the input and output projections will be learned during training. 

Specifically, the input geometry of the stellarator plasma is converted from Fourier coefficientes defining its boundaries into a 3D representation, and the output is the predicted max elongation of the plasma.

The dataloader class is provided in `constellaration_dataloader.py1, and the model is described below. 

In [ ]:
import torch
import torch.nn as nn


class TFNOSurrogate(nn.Module):
    def __init__(
        self,
        input_channels=3,
        output_dim=3,
        hidden_dim=256,
        target_input_channels=28,
        target_output_channels=7,
    ):
        super().__init__()
        self.input_channels = input_channels
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.target_input_channels = target_input_channels
        self.target_output_channels = target_output_channels

        # Input projection: 3D conv to project from input_channels to target_input_channels
        # This projects the 3D volume to match the pretrained TFNO's expected input
        self.input_projection = nn.Sequential(
            nn.Conv3d(input_channels, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv3d(hidden_dim, hidden_dim // 2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv3d(hidden_dim // 2, target_input_channels, kernel_size=1),
        )

        # Output projection: 3D conv to project from target_output_channels to output_dim
        # This projects the TFNO output to our target output size
        self.output_projection = nn.Sequential(
            nn.Conv3d(target_output_channels, hidden_dim // 2, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv3d(hidden_dim // 2, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv3d(
                hidden_dim, 1, kernel_size=1
            ),  # Single channel for global pooling
        )

        # Global pooling layer to convert 3D output to 1D
        self.global_pool = nn.AdaptiveAvgPool3d(1)

        # Final projection to output dimension
        self.final_projection = nn.Sequential(
            nn.Linear(1, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, output_dim),
        )

        self.surrogate = self.setup_model()  # pretrained TFNOPhysicsNeMoModel
        self.freeze_surrogate_weights()

    def setup_model(self):
        """Setup the pretrained TFNO model from The Well as a PhysicsNeMo model.
        Model input parameters are set from the well, and the original config is updated to reflect this."""
        import inspect

        from physicsnemo.models.meta import ModelMetaData
        from physicsnemo.models.module import Module
        from the_well.benchmark.models import TFNO

        well_model = TFNO.from_pretrained("polymathic-ai/TFNO-MHD_64")
        model_dict = well_model.__dict__

        signature = inspect.signature(TFNO)
        parameters = signature.parameters
        filtered_params = {k: model_dict[k] for k in parameters if k in model_dict}

        model = Module.from_torch(TFNO, meta=ModelMetaData(name="converted_tfno"))
        well_pretrained_model = model(**filtered_params)
        well_pretrained_model.inner_model.load_state_dict(
            well_model.state_dict(), strict=True
        )
        return well_pretrained_model

    def freeze_surrogate_weights(self):
        """Freeze all parameters in the pretrained TFNO surrogate model."""
        for param in self.surrogate.parameters():
            param.requires_grad = False

        # Also freeze the surrogate model itself to prevent any updates
        self.surrogate.eval()  # Set to evaluation mode

        print(
            f"Frozen {sum(p.numel() for p in self.surrogate.parameters())} parameters in pretrained TFNO model"
        )

    def forward(self, x):
        # Input: [batch_size, channels, height, width, depth] from FullConstellarationDataset
        # Expected: [batch_size, 3, 64, 64, 64] -> [batch_size, 28, 64, 64, 64]
        # Input projection: 3D conv to match pretrained TFNO input channels
        x_projected = self.input_projection(x)  # [batch_size, 28, 64, 64, 64]

        # Pass through the pretrained TFNO model
        tfno_output = self.surrogate(x_projected)  # [batch_size, 7, 64, 64, 64]

        # Output projection: 3D conv to prepare for global pooling
        tfno_projected = self.output_projection(
            tfno_output
        )  # [batch_size, 1, 64, 64, 64]

        # Global average pooling: [batch_size, 1, 64, 64, 64] -> [batch_size, 1, 1, 1, 1]
        pooled = self.global_pool(tfno_projected)  # [batch_size, 1, 1, 1, 1]

        # Flatten: [batch_size, 1, 1, 1, 1] -> [batch_size, 1]
        flattened = pooled.squeeze(-1).squeeze(-1).squeeze(-1)  # [batch_size, 1]

        # Final projection to output dimension: [batch_size, 1] -> [batch_size, output_dim]
        output = self.final_projection(flattened)  # [batch_size, 3]

        return output

The model and dataloader can then be used with the previous training utilities and a few slight modifications.

In [ ]:
from typing import List
from einops import rearrange


class ConstellarationTrainer(Trainer):
    def setup_model(self):
        """Setup the TFNO model based in PhysicsNeMo. Input parameters are set and controlled by the yaml config."""
        from constellaration_surrogate import TFNOSurrogate

        self.model = TFNOSurrogate(input_channels=3, output_dim=1)
        self.model.to(self.dist.device)

    def _setup_data(self):
        from constellaration_dataloader import ConstellarationDataLoader

        self.datamodule = ConstellarationDataLoader(
            challenge="challenge_1",
            dataset_type="full",
            grid_size=(64, 64, 64),
            batch_size=2,
        )
        self.datamodule.prepare()

    def _forward_pass(self, batch: List[torch.Tensor]) -> torch.Tensor:
        # Add dimension for our one input field
        inputs = batch["input_fields"].unsqueeze(-1)
        # Rearrange for model: (batch, time, x, y, z, fields) -> (batch, (time fields), x, y, z)
        model_inputs = rearrange(inputs, "B Ti Lx Ly Lz F -> B (Ti F) Lx Ly Lz")

        # Forward pass through model
        model_outputs = self.model(model_inputs)
        return model_outputs


cfg.train_params.ckpt_path = "./checkpoints/well_model_constellar_data"
constellaration_model_trainer = ConstellarationTrainer(cfg)
constellaration_model_trainer.train()

## Expanding The Scope - More Models
